# Introduction

In this snippet, we will explore something that does not really fall into the realm of traditional actuarial problems, but still extremely useful nonetheless. This article will go through the process of text classification. <br><br> The dataset used can be found at: <br>
https://www.kaggle.com/sebastienverpile/consumercomplaintsdata <br>
https://catalog.data.gov/dataset/consumer-complaint-database <br>
and consists of finance related complaints that a company has recieved from its customers. <br><br> This problem will be split into 2 parts; transforming text data into usable inputs, and performing classification with those inputs.

The packages that we will be using are listed below. <br>
Pandas and Numpy for general data manipulation <br>
Matplotlib and Seaborn for general data visualisation <br>
Sci-kit learn packages for both feature extraction and classification model

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Part 0 - Importing and Cleaning Data

Since we are trying to predict the category of products based on the narrative of the complaints recieved, we can ignore the rest of the columns for the purpose of this exercise and only look at the single feature and target columns. We will also get rid of null entries as they will not be of any use to us.

In [ ]:
df = pd.read_csv('Consumer_Complaints.csv')
df = df.loc[(df['Consumer complaint narrative'].notnull()), ['Consumer complaint narrative', 'Product']] \
       .reset_index() \
       .drop('index', axis = 1)
df = df[(np.logical_not(df.Product.str.contains(','))) & (df.Product != 'Credit card or prepaid card')]
df.columns = ['description', 'target']

df.head()

Next, we will assign each target variable an integer value instead of using the string, which allows our models to be able to read our responses. We can do this using a variety of methods, but here we will be using sklearn's LabelEncoder function.

In [ ]:
encoder = LabelEncoder()
encoder.fit(df.target)
df = df.assign(encoded_response = lambda x: encoder.transform(x.target))
df[['target', 'encoded_response']].drop_duplicates() \
                                  .set_index('target') \
                                  .sort_values('encoded_response', ascending = True)

We can see from the above output that our resulting dataset contains 14 unique categories which we will try to classify complaints into by training our model to "understand" the narrative.

Next, we will split our dataset for training and validation purposes. Regardless of technique used, train test splits are always considered to be good practice and is extremely helpful to prevent over-fitting of the model.

In [ ]:
x_train, x_test, y_train, y_test, indices_train, indices_test = train_test_split(df.description, 
                                                                                 df.encoded_response,
                                                                                 df.index,
                                                                                 test_size = 0.2, 
                                                                                 random_state = 123)

# Part 1 - Text to features

Although there are many examples of machine learning models that make use of a variety of inputs such as; images ect. we must understand that ultimately, whatever input chosen must be converted into numerical features in order to be "understood" by the model. That being said, images are made up of pixels, which can generally be represented by a integer triplet (x, y, z) indicating the intensity of RGB respectively. <br> In this section, we will find a meaningful way to represent our text data as numerical features. This is also known as feature extraction.

## Bag Of Words Model

One way of transforming a document full of text into numerical features is using the BOW (Bag Of Words) model. Put simply, all it does is assign each word (or token) an ID number, with its respective count. For example, if we have a document string <br>
"This is a cat. That is a dog."
<br>
The BOW representation would simply be {"This": 1, "is": 2, "a": 2, "cat" : 1, "That" : 1, "dog" : 1}
<br>
From the example above we can see that the BOW model just represents a document as its individual component words with its respective count attached to it. Notice that the document shown above is clearly about a cat and a dog. However, our BOW model shows that the most frequent words present in the document is "is" and "a". This is where the importance of data preprocessing is more obvious compared to other machine learning techniques. These common words are also known as "stop words" and are usually taken out of the BOW model to prevent overpowering the words that have actual importance. There are many different little tricks and techniques for choosing the most suitable bag of words to represent your documents and most can be implemented simply through a line (or two) of code using [regular expressions](https://docs.python.org/2/library/re.html).<br>
How does this link to having a numerical matrix as an input? Well, imagine that we have a thousand text documents. We would choose (or use all of) the set of unique words present in that 1000 documents and give word a unique ID number, and make the our columns for the input matrix. For example, if we had chosen 15,000 unique tokens from the 1000 documents as our "dictionary", our input matrix would then have 15,000 columns! Each individual document will then represent a row in the (now extremely wide) matrix filled mostly with zeros, except for the columns corresponding to words found in that particular document, which will contain the count of that word. This technique is also commonly known as count vectorizing. <br>
In the follow example however, we will be using a more robust model compared to count vectorizing called (TF-IDF) Term Frequency - Inverse Document Frequency, and it is defined as:

$w_{i,j} = tf_{i,j} * log(\dfrac{N}{df_i})$ 
<br><br>
$w_{i,j}$ = Weight for word(i) in document(j) <br>
$tf_{i,j}$ = Count of word(i) in document(j) <br>
$N$ = Total number of documents <br>
$df_i$ = How many documents word(i) appears in<br><br>

We can see that, the first term of the TF-IDF model is just the count of the word in the document as before. The magic happens in the second term where the model imposes an additional condition for a particular word to be deemed "important". Just as an example, if the word "bank" appears in every single document, it wouldn't be of much use in differentiating the documents, and the second term of the TF-IDF model expresses this by reducing the whole weight down to 0.

There are a variety of packages that help to automate the vectorizing process but we have chosen to use the Sci-kit learn API due to its easy of usage and interpretability. Here, we instantiate a TFIDF model, with a few note-worthy details: <br>
1) Sublinear_tf uses a logarithmic form of frequency as 20 occurrences of a word probably does not imply 20 times of importance <br>
2) The model will ignore words that appear in less than 5 documents, as well as more than 70% of the total documents <br>
3) Normalization is set to l2 so that all vectors are scaled to have a magnitude of 1 (The equivalent of standardizing your features) <br>
4) There is a layer of preprocessing to remove numerical characters and symbols within the documents through the use of a regular expression

In [ ]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf = True, 
                                   stop_words = 'english',
                                   min_df = 5,
                                   max_df = 0.7,
                                   norm = 'l2',
                                   token_pattern = r'\b[^_\d\W]+\b')

We then use the model to fit our original text data, and convert all that sweet text information into something numerical!

In [ ]:
tfidf_train = tfidf_vectorizer.fit_transform(x_train.values)
tfidf_test = tfidf_vectorizer.transform(x_test.values)

In [ ]:
print(len(tfidf_vectorizer.get_feature_names()))

We can see that our resulting dictionary consists of 19,344 different unqiue words!

In [ ]:
tfidf_df = pd.DataFrame(tfidf_train.A, 
                        columns = tfidf_vectorizer.get_feature_names())

In [ ]:
tfidf_df.head()

As mentioned above, the 19,344 words chosen to be in our "dictionary" become the columns of our input matrix and as you can probably guess, since we have so many columns and each row only consists of words within 1 document, the resulting matrix will be extremely sparse! (Consisting mostly of zeros) <br>
Now that we have extracted some numerical features from our dataset, it is time to use them to train a classifier!

# Part 2 - Classification Model

In this example, we have chosen to use a basic [logistic regression model](http://www.appstate.edu/~whiteheadjc/service/logit/intro.htm) to classify the documents due to tractability and convention. However, more complex models such as neural networks, decision trees, naive bayesian classifiers or other relevant models can also be used to do the following classification.

Firstly, we instantiate a logistic regression classifier from the sklearn package and proceed to fit our vectorized matrix and our encoded training labels.

In [ ]:
lr_classifier = LogisticRegression()
lr_classifier.fit(tfidf_train, y_train)

In [ ]:
pred = lr_classifier.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print(score)

By running the model to predict the categories on the validation set, we can see that we attained some pretty decent predictive power with an overall accuracy of 85.2%. We can drill down into this result by evaluating the [confusion matrix](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology).

In [ ]:
cm = metrics.confusion_matrix(y_test, pred)

In [ ]:
plt.figure(figsize = (15,15))
sns.heatmap(cm, 
            annot = True, 
            fmt = ".0f", 
            linewidths = 0.5, 
            square = True, 
            cmap = 'Blues_r', 
            xticklabels = encoder.classes_,
            yticklabels = encoder.classes_)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size = 15)

One of the obvious results we can observe from the matrix is that "Other financial service", "Vehicle loan or lease" and "Virtual currency" categories had no correct predictions. We must note however, that the number of datapoints we had for each of those categories were insignificant compared to the others and the model probably treated those instances as errors instead of a distinct category.

This bias however, can be corrected by techniques such as [over and undersampling](https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis). Categories with more data points generally had a better score, and errors observed in classification seem to be pretty reasonable due to some overlap between the categories. (For example, 320 descriptions that were supposed to be "Checking and savings account" related were wrongly classified as "Bank account and service".

In [ ]:
def report2dict(cr):
    '''Function purely for formatting purposes, can be ignored'''
    
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    measures = tmp[0]
    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
            
    return D_class_data


results = report2dict(metrics.classification_report(y_test, pred, target_names = encoder.classes_))
pd.DataFrame(results).transpose()

Let's take a look at some examples where 'Checking and savings account' narratives were wrongly classified as 'Bank account and service'.

In [ ]:
def observe_errors(actual_response, wrongly_predicted_response):
    compare = pd.DataFrame(list(zip(x_test, y_test, pred)), columns = ['description', 'actual', 'predicted'])
    compare = compare.assign(actual_product = encoder.inverse_transform(compare.actual),
                             predicted_product = encoder.inverse_transform(compare.predicted)) \
                     .loc[(compare.actual == actual_response) & (compare.predicted == wrongly_predicted_response),
                         ['description', 'actual_product', 'predicted_product']]
    return compare

In [ ]:
observe_errors(actual_response = 1, wrongly_predicted_response = 0).tail(5)

We can the misclassified complaints shown above is quite ambiguous and contain ideas and keywords within the narrative related to both categories, which is presumably why the model would have misclassified them.

# Afterward

As discussed above, data preprocessing is an extremely important step when it comes to NLP problems and the "garbage-in, garbage-out" property can be very prevalent compared to other machine learning techniques. Also note that, in this problem that there are existing labels in the dataset, and we could have just as easily used any other supervised learning techniques to classify the documents. If however, no labels are available, we would then have to turn towards unsupervised learning algorithms such as SVM, K-means or LDA to try and cluster the documents into sensible categories.

We have essentially just taught the computer how to "understand" a small selected group of documents in a very human way, which is just amazing! So, you might ask, how does this apply to actuarial work? <br>
Imagine having access to all the historical claim narratives within your company's database. What do you think are some possible projects that will add value to your insurer now that you know how to model text? Could you maybe come up with a classifier using techniques above coupled with some clustering analysis techniques such as K-Means to categorize the claims if the legacy system does not have this feature? What about doing some sentiment analysis on the claim narratives to get the general satisfaction level of customers and to help pinpoint the strengths and weaknesses of the insurer's current claims handling process? <br>
Do not be mistaken though, the possibilites are not constrained to the claims management section of the insurance value chain. Other examples include chatbots and recommendation systems as distribution channels, and fraud detection with text analysis. The possibilities are ever-growing, technology ever-improving and it is up to the actuarial profession to keep up with the trends in order to continue adding value to the industry.


# Additional Resources

[TF-IDF Model](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) <br>
[Python Regular Expressions (REGEX)](https://www.tutorialspoint.com/python/python_reg_expressions.htm) <br>
[Logistic Regression](http://www.appstate.edu/~whiteheadjc/service/logit/intro.htm) <br>
[Confusion Matrix](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/)